In [ ]:
import requests
from bs4 import BeautifulSoup
import json
import csv

def GetNewsInfo():
    headers = {
        'authority': 'news.kbs.co.kr',
        'sec-ch-ua': '" Not A;Brand";v="99", "Chromium";v="90", "Google Chrome";v="90"',
        'accept': 'application/json, text/javascript, */*; q=0.01',
        'x-requested-with': 'XMLHttpRequest',
        'sec-ch-ua-mobile': '?0',
        'user-agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.72 Safari/537.36',
        'content-type': 'application/x-www-form-urlencoded; charset=UTF-8',
        'origin': 'https://news.kbs.co.kr',
        'sec-fetch-site': 'same-origin',
        'sec-fetch-mode': 'cors',
        'sec-fetch-dest': 'empty',
        'referer': 'https://news.kbs.co.kr/news/list.do?ctcd=0004&ref=pMenu',
        'accept-language': 'en-US,en;q=0.9,ko;q=0.8',
        'cookie': 'newsUserToken=vgjoUAyhcmFCVBjP7DVnqg8qyL4vGYa66raFSAzqdlbLn0pVXhvQjAWpHZPc6PHz2BIaR4Ikl5TTaCrSmp9GJlPX1wYjZ8aVnE8U; _ga=GA1.3.762818420.1624600426; _gid=GA1.3.753340869.1624600426; STAT_WEBLOG_TOKEN=5945d3f0-80df-49ee-8109-79df85766ed6; gtm-session-start=1624606539548; STAT_WEBLOG_SESSION_ID=cc9fa545-aa1e-4609-b8ec-2d2e3f990253',
    }

    news_info=[]
    no = 1

    # 1페이지에 12개의 아티클이 있음, 1일 48개정도 아티클이 올라옴
    for num in range(1, 5):
        data = {
        'currentPageNo': f'{num}',
        'rowsPerPage': '12',
        'exceptPhotoYn': 'Y',
        'datetimeBegin': '20210625000000',
        'datetimeEnd': '20210625235959',
        'contentsCode': '0004',
        'localCode': '00'
        }

        url = "https://news.kbs.co.kr/api/getNewsList"
        response = requests.post(url, headers=headers, data=data)

        datas = response.json()['data']

        # api 파싱해서 뉴스코드, 뉴스제목, 카테고리. 이미지 주소 가져오는 부분
        for data in datas:
            news_code = data['newsCode']
            category = data['contentsName']
            date = data['regDate']
            title = data['newsTitle']
            img_url = data['imgUrl']

            # 뷰티풀숲으로 언론사, 뉴스 내용 가져오는 부분
            url = f'https://news.kbs.co.kr/news/view.do?ncd={news_code}'
            res = requests.get(url)
            soup = BeautifulSoup(res.text, 'html.parser')

            press = soup.select_one('#header > div.header-info > div.header-gnb > div > h1 > a > span.blind').text
            content = soup.select_one('#cont_newstext').text.strip()

            info = {
                'no' : no,
                'press' : press,
                'news_code' : news_code,
                'category' : category,
                'date' : date,
                'title' : title,
                'content' : content,
                'img' : img_url,
                'ref' : url
            }
            news_info.append(info)
            no += 1
            
    return news_info

news_info = GetNewsInfo()

with open('./news_info.csv', 'w', newline='') as c:
    fieldnames = ['no', 'press', 'news_code', 'category', 'date', 'title', 'content', 'img', 'ref' ]
    writer = csv.DictWriter(c, fieldnames=fieldnames) 
    writer.writeheader()

    for news in news_info:
        writer.writerow(news)